In [1]:
import os
import glob
import shutil
import zipfile
import json
import pandas as pd
import numpy as np
from pandas import DataFrame

from datetime import datetime

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import os
import json
import pandas as pd


import matplotlib as mlp
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname = "C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family = font_name)
mlp.rcParams["axes.unicode_minus"] = False

## 경로 설정

In [2]:
ROOT_DIR = 'E:/'

YOLOv5_ROOT_DIR = os.path.join(ROOT_DIR, 'yolov5')

DATA_ROOT_DIR = os.path.join(ROOT_DIR, 'rawdata_disease_resize')
HOUSE_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, 'house')

In [3]:
TRAIN_ROOT_DIR = os.path.join(HOUSE_DATA_ROOT_DIR, 'train')
VALID_ROOT_DIR = os.path.join(HOUSE_DATA_ROOT_DIR, 'validation')

## YOLO source 가져오기

In [4]:
!rm -rf {YOLOv5_ROOT_DIR}

!cd {ROOT_DIR}
!git clone https://github.com/ultralytics/yolov5

'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
fatal: destination path 'yolov5' already exists and is not an empty directory.


## 필수 라이브러리 설치

In [3]:
!pip install -r {os.path.join(YOLOv5_ROOT_DIR, 'requirements.txt')}

Defaulting to user installation because normal site-packages is not writeable


## pre-trained model 다운로드

In [5]:
!wget -P /yolov5/ https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


# train에서

## images, labels 폴더 생성

In [8]:
if not os.path.exists(os.path.join(TRAIN_ROOT_DIR, 'images')):
    os.mkdir(os.path.join(TRAIN_ROOT_DIR, 'images'))
    
if not os.path.exists(os.path.join(TRAIN_ROOT_DIR, 'labels')):
    os.mkdir(os.path.join(TRAIN_ROOT_DIR, 'labels'))

## 필요한 파일 images, labels 폴더로 나눠서 압축 풀기

In [9]:
os.listdir(TRAIN_ROOT_DIR)

['image_tomato_0.normal.zip',
 'image_tomato_1.disease.zip',
 'image_tomato_9.disease(1).zip',
 'image_tomato_9.disease(2).zip',
 'imgaes',
 'labels',
 'label_tomato_0.normal.zip',
 'label_tomato_1.disease.zip',
 'label_tomato_9.disease.zip']

In [12]:
with zipfile.ZipFile(os.path.join(TRAIN_ROOT_DIR, 'image_tomato_0.normal.zip'), 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.split("_")[5] == '03':
            zip_ref.extract(file, os.path.join(TRAIN_ROOT_DIR, 'images', 'image_tomato_0.normal'))

In [11]:
with zipfile.ZipFile(os.path.join(TRAIN_ROOT_DIR, 'image_tomato_9.disease(1).zip'), 'r') as target_file:
    target_file.extractall(os.path.join(TRAIN_ROOT_DIR, 'images', 'image_tomato_9.disease'))

In [13]:
with zipfile.ZipFile(os.path.join(TRAIN_ROOT_DIR, 'image_tomato_9.disease(2).zip'), 'r') as target_file:
    target_file.extractall(os.path.join(TRAIN_ROOT_DIR, 'images', 'image_tomato_9.disease'))

In [16]:
with zipfile.ZipFile(os.path.join(TRAIN_ROOT_DIR, 'label_tomato_0.normal.zip'), 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.split("_")[5] == '03':
            zip_ref.extract(file, os.path.join(TRAIN_ROOT_DIR, 'labels', 'label_tomato_0.normal'))

In [15]:
with zipfile.ZipFile(os.path.join(TRAIN_ROOT_DIR, 'label_tomato_9.disease.zip'), 'r') as target_file:
    target_file.extractall(os.path.join(TRAIN_ROOT_DIR, 'labels', 'label_tomato_9.disease'))

# validation에서

## images, labels 폴더 생성

In [18]:
if not os.path.exists(os.path.join(VALID_ROOT_DIR, 'images')):
    os.mkdir(os.path.join(VALID_ROOT_DIR, 'images'))
    
if not os.path.exists(os.path.join(VALID_ROOT_DIR, 'labels')):
    os.mkdir(os.path.join(VALID_ROOT_DIR, 'labels'))

## 필요한 파일 images, labels 폴더로 나눠서 압축 풀기

In [19]:
os.listdir(VALID_ROOT_DIR)

['11.토마토_0.정상.zip',
 '11.토마토_1.질병.zip',
 '11.토마토_9.증강.zip',
 'images',
 'labels',
 '[라벨]11.토마토_0.정상.zip',
 '[라벨]11.토마토_1.질병.zip',
 '[라벨]11.토마토_9.증강.zip']

In [20]:
with zipfile.ZipFile(os.path.join(VALID_ROOT_DIR, '11.토마토_0.정상.zip'), 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.split("_")[5] == '03':
            zip_ref.extract(file, os.path.join(VALID_ROOT_DIR, 'images', 'image_tomato_0.normal'))

In [21]:
with zipfile.ZipFile(os.path.join(VALID_ROOT_DIR, '11.토마토_9.증강.zip'), 'r') as target_file:
    target_file.extractall(os.path.join(VALID_ROOT_DIR, 'images', 'image_tomato_9.disease'))

In [22]:
with zipfile.ZipFile(os.path.join(VALID_ROOT_DIR, '[라벨]11.토마토_0.정상.zip'), 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.split("_")[5] == '03':
            zip_ref.extract(file, os.path.join(VALID_ROOT_DIR, 'labels', 'label_tomato_0.normal'))

In [23]:
with zipfile.ZipFile(os.path.join(VALID_ROOT_DIR, '[라벨]11.토마토_9.증강.zip'), 'r') as target_file:
    target_file.extractall(os.path.join(VALID_ROOT_DIR, 'labels', 'label_tomato_9.disease'))

---

# 데이터 살펴보기

In [29]:
df = pd.read_csv('D:/tomato/disease/total_df.csv', index_col=0)
df.head(3)

,image,date,worker,height,width,task,type,region,disease,crop,area,grow,risk,points,xtl,ytl,xbr,ybr,file
0,V006_77_0_00_11_01_13_0_c01_20201013_0000_S01.jpg,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr'...",1864,1516,2292,1880,0
1,V006_77_0_00_11_01_13_0_c01_20201013_0000_S01_...,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 1722, 'ytl': 436, 'xbr': 2167, 'ybr':...",1722,436,2167,676,0
2,V006_77_0_00_11_01_13_0_c01_20201013_0001_S01.jpg,2020/10/13,NaN,3024,4032,77,0,NaN,0,11,1,13,0,"[{'xtl': 2150, 'ytl': 100, 'xbr': 2727, 'ybr':...",2150,100,2727,784,0


In [31]:
df.risk.value_counts()

0    13509
2    12443
1     5468
3     3086
Name: risk, dtype: int64

# validation에서

## json

In [5]:
with open('D:/rawdata_disease/house/validation/labels/label_tomato_0.normal/V006_77_0_00_11_03_12_0_c22_20201130_0001_S01_2.jpg.json') as json_file:
    json_data = json.load(json_file)

json_data

{'description': {'image': 'V006_77_0_00_11_03_12_0_c22_20201130_0001_S01_2.jpg',
  'date': '2020/11/30',
  'worker': '',
  'height': 4032,
  'width': 3024,
  'task': 77,
  'type': 0,
  'region': None},
 'annotations': {'disease': 0,
  'crop': 11,
  'area': 3,
  'grow': 12,
  'risk': 0,
  'points': [{'xtl': 148, 'ytl': 1129, 'xbr': 2807, 'ybr': 3338}]}}

## labels 폴더 내 txt파일 저장 폴더 생성

In [6]:
if not os.path.exists(os.path.join(VALID_ROOT_DIR, 'labels', 'label_tomato_0.normal_txt')):
    os.mkdir(os.path.join(VALID_ROOT_DIR, 'labels', 'label_tomato_0.normal_txt'))
    
if not os.path.exists(os.path.join(VALID_ROOT_DIR, 'labels', 'label_tomato_9.disease_txt')):
    os.mkdir(os.path.join(VALID_ROOT_DIR, 'labels', 'label_tomato_9.disease_txt'))

## json to txt

In [9]:
def convert_json_to_yolo(json_data):
    
    with open(json_file_path) as json_file:
        json_data = json.load(json_file)
    
    # 이미지 정보 및 경로 추출
    image_path = json_data["description"]["image"]
    image_width = json_data["description"]["width"]
    image_height = json_data["description"]["height"]
    
    # YOLO 형식으로 변환할 내용 추출
    class_id = json_data["annotations"]["risk"]
    xtl = json_data["annotations"]["points"][0]["xtl"]
    ytl = json_data["annotations"]["points"][0]["ytl"]
    xbr = json_data["annotations"]["points"][0]["xbr"]
    ybr = json_data["annotations"]["points"][0]["ybr"]
    
    # YOLO 좌표 계산 및 정규화
    bbox_width = xbr - xtl
    bbox_height = ybr - ytl
    x_center = (xtl + xbr) / 2 / image_width
    y_center = (ytl + ybr) / 2 / image_height
    bbox_width = bbox_width / image_width
    bbox_height = bbox_height / image_height
    
    # YOLO 형식의 문자열 생성
    yolo_string = f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}"
    
    return image_path, yolo_string

# JSON 파일이 있는 디렉토리 경로
json_directory = "D:/rawdata_disease/house/validation/labels/"

# 변환된 YOLO 파일을 저장할 디렉토리 경로
txt_directory = "D:/rawdata_disease/house/validation/labels/"

# 디렉토리 내의 모든 JSON 파일 처리
for filename in os.listdir(json_directory):
    json_file_path = os.path.join(json_directory, filename)
        
    # JSON을 YOLO 형식으로 변환
    image_path, yolo_string = convert_json_to_yolo(json_file_path)
        
    # 변환된 내용을 텍스트 파일로 저장
    txt_file_path = os.path.join(txt_directory, f"{os.path.splitext(filename)[0]}.txt")
    with open(txt_file_path, "w") as txt_file:
        txt_file.write(yolo_string)

# train에서

## labels 폴더 내 txt파일 저장 폴더 생성

In [10]:
if not os.path.exists(os.path.join(TRAIN_ROOT_DIR, 'labels', 'label_tomato_0.normal_txt')):
    os.mkdir(os.path.join(TRAIN_ROOT_DIR, 'labels', 'label_tomato_0.normal_txt'))
    
if not os.path.exists(os.path.join(TRAIN_ROOT_DIR, 'labels', 'label_tomato_9.disease_txt')):
    os.mkdir(os.path.join(TRAIN_ROOT_DIR, 'labels', 'label_tomato_9.disease_txt'))

## json to txt

In [12]:
def convert_json_to_yolo(json_data):
    
    with open(json_file_path) as json_file:
        json_data = json.load(json_file)
    
    # 이미지 정보 및 경로 추출
    image_path = json_data["description"]["image"]
    image_width = json_data["description"]["width"]
    image_height = json_data["description"]["height"]
    
    # YOLO 형식으로 변환할 내용 추출
    class_id = json_data["annotations"]["risk"]
    xtl = json_data["annotations"]["points"][0]["xtl"]
    ytl = json_data["annotations"]["points"][0]["ytl"]
    xbr = json_data["annotations"]["points"][0]["xbr"]
    ybr = json_data["annotations"]["points"][0]["ybr"]
    
    # YOLO 좌표 계산 및 정규화
    bbox_width = xbr - xtl
    bbox_height = ybr - ytl
    x_center = (xtl + xbr) / 2 / image_width
    y_center = (ytl + ybr) / 2 / image_height
    bbox_width = bbox_width / image_width
    bbox_height = bbox_height / image_height
    
    # YOLO 형식의 문자열 생성
    yolo_string = f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}"
    
    return image_path, yolo_string

# JSON 파일이 있는 디렉토리 경로
#json_directory = "D:/rawdata_disease/house/train/labels/label_tomato_9.disease/"
json_directory = "D:/rawdata_disease/house/train/labels/label_tomato_0.normal/"

# 변환된 YOLO 파일을 저장할 디렉토리 경로
#txt_directory = "D:/rawdata_disease/house/train/labels/label_tomato_9.disease_txt/"
txt_directory = "D:/rawdata_disease/house/train/labels/label_tomato_0.normal_txt/"

# 디렉토리 내의 모든 JSON 파일 처리
for filename in os.listdir(json_directory):
    json_file_path = os.path.join(json_directory, filename)
        
    # JSON을 YOLO 형식으로 변환
    image_path, yolo_string = convert_json_to_yolo(json_file_path)
        
    # 변환된 내용을 텍스트 파일로 저장
    txt_file_path = os.path.join(txt_directory, f"{os.path.splitext(filename)[0]}.txt")
    with open(txt_file_path, "w") as txt_file:
        txt_file.write(yolo_string)

## train/valid image와 label 개수 및 파일명 일치여부 체크

In [4]:
TRAIN_IMAGE_DATA_DIR = os.path.join(HOUSE_DATA_ROOT_DIR, 'train/images/')
TRAIN_LABEL_DATA_DIR = os.path.join(HOUSE_DATA_ROOT_DIR, 'train/labels/')

VALID_IMAGE_DATA_DIR = os.path.join(HOUSE_DATA_ROOT_DIR, 'validation/images/')
VALID_LABEL_DATA_DIR = os.path.join(HOUSE_DATA_ROOT_DIR, 'validation/labels/')

In [14]:
#-----------변수 초기화----------
train_image_file_list = []
train_label_file_list = []

train_image_file_name_list = []
train_label_file_name_list = []

train_image_file_name_set = set()
train_label_file_name_set = set()
#---------------------------------

train_image_file_list = os.listdir(TRAIN_IMAGE_DATA_DIR)
train_label_file_list = os.listdir(TRAIN_LABEL_DATA_DIR)

train_image_file_list.sort()
train_label_file_list.sort()

# image file name 추출하기
for index in range(len(train_image_file_list)):
    image_name = os.path.splitext(train_image_file_list[index])[0]
    train_image_file_name_list.append(image_name)
    
# set -> 중복제거 & 집합연산 가능
train_image_file_name_set = set(train_image_file_name_list)

# label file name 추출하기
for index in range(len(train_label_file_list)):
    label_name = os.path.splitext(train_label_file_list[index])[0].split('.')[0]
    train_label_file_name_list.append(label_name)

train_label_file_name_set = set(train_label_file_name_list)

# image file과 label file 비교
diff_set = train_image_file_name_set - train_label_file_name_set
intersect_set = train_image_file_name_set & train_label_file_name_set

if len(diff_set) == 0 :
    print('>>> image is exactly same as label <<<')
else:
    print('>>>different<<<')
    print(len(diff_set))

>>> image is exactly same as label <<<


In [8]:
train_image_file_list = os.listdir(TRAIN_IMAGE_DATA_DIR)
train_image_file_list[0]

'V006_77_0_00_11_03_12_0_c01_20201013_0000_S01.JPG'

In [10]:
os.path.splitext(train_image_file_list[10000])

('V006_77_1_18_11_03_13_2_9008w_20201023_127_a0008', '.jpg')

In [11]:
train_label_file_list = os.listdir(TRAIN_LABEL_DATA_DIR)
train_label_file_list[0]

'V006_77_0_00_11_03_12_0_c01_20201013_0000_S01.JPG.txt'

In [13]:
os.path.splitext(train_label_file_list[0])[0].split('.')[0]

'V006_77_0_00_11_03_12_0_c01_20201013_0000_S01'

In [19]:
#-----------변수 초기화----------
valid_image_file_list = []
valid_label_file_list = []

valid_image_file_name_list = []
valid_label_file_name_list = []

valid_image_file_name_set = set()
valid_label_file_name_set = set()
#---------------------------------

valid_image_file_list = os.listdir(VALID_IMAGE_DATA_DIR)
valid_label_file_list = os.listdir(VALID_LABEL_DATA_DIR)

valid_image_file_list.sort()
valid_label_file_list.sort()

# image file name 추출하기
for index in range(len(valid_image_file_list)):
    image_name = os.path.splitext(valid_image_file_list[index])[0]
    valid_image_file_name_list.append(image_name)
    
# set -> 중복제거 & 집합연산 가능
valid_image_file_name_set = set(valid_image_file_name_list)

# label file name 추출하기
for index in range(len(valid_label_file_list)):
    label_name = os.path.splitext(valid_label_file_list[index])[0].split('.')[0]
    valid_label_file_name_list.append(label_name)

valid_label_file_name_set = set(valid_label_file_name_list)

# image file과 label file 비교
diff_set = valid_image_file_name_set - valid_label_file_name_set
intersect_set = valid_image_file_name_set & valid_label_file_name_set

if len(diff_set) == 0 :
    print('>>> image is exactly same as label <<<')
else:
    print('>>>different<<<')
    print(len(diff_set))

>>> image is exactly same as label <<<


# label 파일명 수정

- train

예시

In [13]:
os.listdir(TRAIN_LABEL_DATA_DIR)[5]

'V006_77_0_00_11_03_12_0_c01_20201013_0002_S01_1.JPG.txt'

In [14]:
os.listdir(TRAIN_LABEL_DATA_DIR)[5].split('.')[0]+'.'+os.listdir(TRAIN_LABEL_DATA_DIR)[5].split('.')[2]

'V006_77_0_00_11_03_12_0_c01_20201013_0002_S01_1.txt'

실제 실행 코드

In [18]:
file_list = os.listdir(TRAIN_LABEL_DATA_DIR)

for i in range(len(file_list)):
    try:
        old_name = file_list[i]
        new_name = old_name.split('.')[0] + '.' + old_name.split('.')[2]
        os.rename(os.path.join(TRAIN_LABEL_DATA_DIR, old_name), os.path.join(TRAIN_LABEL_DATA_DIR, new_name))
    except OSError as e:
        print("파일명 변경에 실패했습니다. 예외 메시지:", str(e), i)

- valid

예시

In [19]:
os.listdir(VALID_LABEL_DATA_DIR)[5]

'V006_77_0_00_11_03_13_0_c05_20201222_0003_S01_2.JPG.txt'

실제 실행 코드

In [20]:
file_list = os.listdir(VALID_LABEL_DATA_DIR)

for i in range(len(file_list)):
    try:
        old_name = file_list[i]
        new_name = old_name.split('.')[0] + '.' + old_name.split('.')[2]
        os.rename(os.path.join(VALID_LABEL_DATA_DIR, old_name), os.path.join(VALID_LABEL_DATA_DIR, new_name))
    except OSError as e:
        print("파일명 변경에 실패했습니다. 예외 메시지:", str(e), i)

# YOLOv8

In [4]:
import ultralytics

In [5]:
ultralytics.checks()

Ultralytics YOLOv8.0.117  Python-3.9.13 torch-2.0.1+cpu CPU
Setup complete  (8 CPUs, 15.9 GB RAM, 227.2/298.1 GB disk)


In [6]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

## YAML 파일 설정

In [7]:
import yaml

with open(os.path.join(HOUSE_DATA_ROOT_DIR, 'data.yaml'), 'r') as f:
    data = yaml.safe_load(f)
    
print(data)

data['train'] = os.path.join(HOUSE_DATA_ROOT_DIR, 'train/images/')
data['val'] = os.path.join(HOUSE_DATA_ROOT_DIR, 'validation/images/')

data['nc'] = 4
data['names'] = ['normal', 'early', 'mid', 'late']

with open(os.path.join(HOUSE_DATA_ROOT_DIR, 'data.yaml'), 'w') as f:
    yaml.dump(data, f)
    
print(data)

{'names': ['normal', 'early', 'mid', 'late'], 'nc': 4, 'train': 'E:/rawdata_disease_resize\\house\\train/images/', 'val': 'E:/rawdata_disease_resize\\house\\validation/images/'}
{'names': ['normal', 'early', 'mid', 'late'], 'nc': 4, 'train': 'E:/rawdata_disease_resize\\house\\train/images/', 'val': 'E:/rawdata_disease_resize\\house\\validation/images/'}


In [8]:
start_time = datetime.now()
print("Start Training :", start_time)
##########


model.train(data="E:/rawdata_disease_resize/house/data.yaml",
            epochs=40, patience=4,
            batch=32, imgsz=416)


##########
end_time = datetime.now()
print("End of Training :", end_time)
print("\nTime taken :", end_time-start_time)

New https://pypi.org/project/ultralytics/8.0.124 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.117  Python-3.9.13 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=E:/rawdata_disease_resize/house/data.yaml, epochs=2, patience=4, batch=32, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=

Start Training : 2023-06-30 11:04:53.148935


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

End of Training : 2023-06-30 14:03:47.922043

Time taken : 2:58:54.773108


In [ ]:
model.save('/content/disease_risk.h5')

In [ ]:
from torch.serialization import save
results = model.predict(source=os.path.join(ROOT_DIR, 'house/test/images'), save=True)